### First test
This is my first MWE of animations + GPS locator on folium

In [15]:
#https://nbviewer.org/github/python-visualization/folium/blob/main/examples/Plugins.ipynb#Timestamped-GeoJSON
from folium import plugins
import folium

circlemarkers = [{"time": "2017-10-03 01:00", 'radius': 5, 'tooltip': 'hi world', "location": [54.677851,25.282656][::-1]},
                 {"time": "2017-10-03 02:00", 'radius': 5, 'tooltip': 'hi world again', "location": [54.6778603,25.2553013][::-1]},
                 {"time": "2017-10-03 03:00", 'radius': 5, 'tooltip': 'hi world aa', "location": [54.6940226,25.2760565][::-1]},
                 {"time": "2017-10-03 04:00", 'radius': 5, 'tooltip': 'hi world kk', "location": [54.73978779999999,25.2746094][::-1]}]

features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": point["location"],
        },
        "properties": {
            "time": point["time"],
            "tooltip": point["tooltip"],
            "id": f"house{i}",
            "icon": "circle",
            "iconstyle": {
                "fillColor": "green",
                "fillOpacity": 0.6,
                "stroke": "true",
                "radius": 7}
        },
    }
    for i, point in enumerate(circlemarkers)
]


m = folium.Map(location=[54.677851,25.282656], tiles='openstreetmap', zoom_start=12)


plugins.TimestampedGeoJson(
    {"type": "FeatureCollection", "features": features},
    period="PT1H",
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options="hh:mm",
    time_slider_drag_update=True,
    duration="PT0H",
).add_to(m)

plugins.LocateControl(auto_start=False, flyTo=True).add_to(m)
m
#the render of straight notebook via nbviewer is terrible on phone

Recover methods from `folium mapping` and new version of `map_restaurant_activity` method in `restaurant_locations` -> `scrapping_populartimes`   
And let's plot first collection of restaurants active at Mon 00:00 with this improved style of `map_restaurant_activity`.

In [2]:
import folium
import pandas as pd
import numpy as np
import os
from IPython.display import IFrame

def name_to_file(restaurant, weekday):
    repl = restaurant.replace('"', '').replace(f'/', '')
    return f'{weekday}/{repl}.png'

def map_restaurant_activity(df, colorsep=[], weekday='Monday'):
    #requires popular_times.npz and folder of weekdays of populartimes snapshots
    route_map = folium.Map(location=(54.690697, 25.279788), zoom_start=14, tiles='openstreetmap')
    nonindexed_groups = list(df.groupby(['query']).indices.values())
    nonindexed_groups = sorted(nonindexed_groups, key=lambda x: x[0])
    groups = [df.index[n] for n in nonindexed_groups] #in case df has missing idx
    npz = np.load('popular_times.npz')
    for idx in groups:
        query, name, address, votes, lat, lng, open, close = df.loc[idx[0]].values
        board_title = f"<b>{query}</b>"
        board_hours = '<br>'.join([f'{x} - {y}' for x,y in df.loc[idx, ['open', 'close']].values])
        board_img_desc = f'<b>{weekday} activity:</b>'
        if query in npz.files:
            board_img = f'<img src="{name_to_file(query, weekday)}" width="200" height="150" frameborder="0">'
        else:
            board_img = f'<img src="{name_to_file("""MEAN""", weekday)}" width="200" height="150" frameborder="0">'
        board_votes = f"VOTES: <b>{int(votes)}</b>"
        board = '<br>'.join([board_title, board_hours, board_votes, board_img_desc, board_img])
        if query in colorsep:
            fill_opacity, fill_color = 0.6, 'darkgreen'
        else:
            fill_opacity, fill_color = 0.5, 'blue'
        folium.CircleMarker(radius=0.2*max(votes, 200)**0.5, 
                            location=(lat, lng), 
                            tooltip=board,
                            color='black', fill_color=fill_color, fill_opacity=fill_opacity,
                            fill=True, max_width=50).add_to(route_map)
        
    return route_map

def save_route_map(route_map, file_name):
    print('CREATE:', file_name)
    route_map.save(file_name)

def openness(A, B):
    interval_normal = A[:, 0] < A[:, 1] #if interval is normal: a<= point <=b; otherwise (point<b) or (point>a)
    since_open = A[:, 0] <= B[:, None] #a <= point
    till_close = A[:, 1] > B[:, None] #b >= point
    normal_case = interval_normal & since_open & till_close
    jump_case = ~interval_normal & (since_open | till_close)
    return normal_case | jump_case

def rtm(x, d = {'Sun': 0, 'Mon': 1, 'Tue': 2, 'Wed': 3, 'Thu': 4, 'Fri': 5, 'Sat': 6}): 
    try:
        #print(x) in case there is mistake in a file
        return d[x[:3]]*1440+int(x[4:6])*60+int(x[7:])
    except ValueError:
        raise ValueError(f'Incorrect date format. {x} must be of the form Day HH:MM')
def dtr(x, d = {0: 'Sun', 1: 'Mon', 2: 'Tue', 3: 'Wed', 4: 'Thu', 5: 'Fri', 6: 'Sat'}): 
    return f"{d[x[0]]} {x[1][:-2].zfill(2)}:{x[1][-2:]}"
    
def readable_to_minutes(X):
    '''Day HH:MM -> min: int'''
    d = {'Sun': 0, 'Mon': 1, 'Tue': 2, 'Wed': 3, 'Thu': 4, 'Fri': 5, 'Sat': 6}
    return X.agg(rtm)
    
def dayntime_to_readable(X):
    '''[day: int, time: str] > Day HH:MM'''
    d = {0: 'Sun', 1: 'Mon', 2: 'Tue', 3: 'Wed', 4: 'Thu', 5: 'Fri', 6: 'Sat'}
    return X.agg(dtr, axis=1)

df = pd.read_csv('bolt_restaurants_vilnius.txt')
moments = np.arange(1440, 1440*8, 60)%(1440*7)
_open, _close = readable_to_minutes(df['open']), readable_to_minutes(df['close'])
business_matrix = openness(np.transpose([_open, _close]), moments)
daymap = {0: 'Sun', 1: 'Mon', 2: 'Tue', 3: 'Wed', 4: 'Thu', 5: 'Fri', 6: 'Sat'}
groups = sorted([y for y in df.groupby(['query']).indices.values()], key=lambda x: x[0])
unique_rows = df.iloc[[y[0] for y in groups]]
colorsep = unique_rows[:228]['query'].values

    
idx_collection = []
for idx, m in zip(business_matrix, moments):
    day, hour, minute = np.unravel_index(m, (7, 24, 60))
    date = f'2000-01-01 {str(hour).zfill(2)}:{str(minute).zfill(2)}' #{daymap[day]}
    rows = df.loc[idx]
    idx_collection.append(idx)
    
#if daymap[day] not in os.listdir(title): os.makedirs(f'{title}/{daymap[day]}')
subdf = df.loc[idx_collection[0]]
route_map = map_restaurant_activity(subdf, colorsep=colorsep, weekday='Monday')
save_route_map(route_map, 'bolt_animation_test.html')
IFrame(src='./bolt_animation_test.html', width=800, height=300)

CREATE: bolt_animation_test.html


In [22]:
subdf

,query,name,address,votes,lat,lng,open,close
6,Bolt Market (Kalvarijų g.),"Bolt Market, Kalvariju","Kalvarijų g. 105, 08219 Vilnius, Lithuania",9,54.708286,25.286013,Sun 09:00,Mon 02:00
14,McDonald’s® (Pilaitė),McDonald's,"Pilaitės pr. 18, 04352 Vilnius, Lithuania",923,54.699191,25.212279,Mon 00:00,Mon 23:59
21,McDonald’s® (Kedrų g.),McDonald's,"Kedrų g. 2A, 03159 Vilnius, Lithuania",2684,54.676424,25.256505,Mon 00:00,Mon 23:59
35,Jammi kebabai (Tauro g.),Jammi,"Tauro g. 3, 03106 Vilnius, Lithuania",3160,54.684230,25.273290,Mon 00:00,Mon 23:59
42,Jammi kebabai (Medeinos g.),Jammi,"Medeinos g. 8, 06112 Vilnius, Lithuania",1324,54.729849,25.227265,Mon 00:00,Mon 23:59
61,Jammi kebabai (Ukmergės g.),Jammi,"Ukmergės g. 253, 07100 Vilnius, Lithuania",1187,54.727528,25.240972,Mon 00:00,Mon 23:59
68,Jammi kebabai (P. Lukšio g.),Jammi,"P. Lukšio g. 11, 09108 Vilnius, Lithuania",590,54.712845,25.291168,Mon 00:00,Mon 23:59
96,Jammi kebabai (Blindžių g.),Jammi,"Blindžių g. 10, 08111 Vilnius, Lithuania",1712,54.698290,25.245201,Mon 00:00,Mon 23:59
110,Bolt Market (Ukmergės g.),"Bolt Market, Ukmergės","Ukmergės g. 220, 07157 Vilnius, Lithuania",5,54.716119,25.246250,Sun 09:00,Mon 01:00
124,Jammi kebabai (Ateities g.),Jammi,"Ateities g. 10, 08303 Vilnius, Lithuania",1387,54.737113,25.267479,Mon 00:00,Mon 23:59


Next, rewrite `mar_restaurant_activity` to return elements in the form: 

    circlemarkers = [{"time": "2017-10-03 01:00", 'radius': 5, 'tooltip': 'hi world', "location": [54.677,25.282][::-1]},
                     {"time": "2017-10-03 02:00", 'radius': 5, 'tooltip': 'hia', "location": [54.677,25.255][::-1]},
                     {"time": "2017-10-03 03:00", 'radius': 5, 'tooltip': 'hi world aa', "location": [54.694,25.276][::-1]},
                     {"time": "2017-10-03 04:00", 'radius': 5, 'tooltip': 'hi world kk', "location": [54.739,25.274][::-1]}]

    features = [
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": point["location"],
            },
            "properties": {
                "time": point["time"],
                "tooltip": point["tooltip"],
                "id": f"house{i}",
                "icon": "circle",
                "iconstyle": {
                    "fillColor": "green",
                    "fillOpacity": 0.6,
                    "stroke": "true",
                    "radius": 7}
            },
        }
        for i, point in enumerate(circlemarkers)
    ]


## Main run!

In [8]:
def activity_circles(date, df, colorsep=[]):
    #requires popular_times.npz and folder of weekdays of populartimes snapshots
    d = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
    day, hour, minute = date
    weekday = d[(day-1)%7]
    date_repr = f'2000-01-01 {str(hour).zfill(2)}:{str(minute).zfill(2)}' #{daymap[day]}
        
    features = []
    route_map = folium.Map(location=(54.690697, 25.279788), zoom_start=14, tiles='openstreetmap')
    nonindexed_groups = list(df.groupby(['query']).indices.values())
    nonindexed_groups = sorted(nonindexed_groups, key=lambda x: x[0])
    groups = [df.index[n] for n in nonindexed_groups] #in case df has missing idx
    npz = np.load('popular_times.npz')
    M = np.mean(np.array([npz[name] for name in npz.files]), axis=0)
    
    for idx in groups:
        query, name, address, votes, lat, lng, open, close = df.loc[idx[0]].values
        board_title = f"<b>{query}</b>"
        board_hours = '<br>'.join([f'{x} - {y}' for x,y in df.loc[idx, ['open', 'close']].values])
        board_img_desc = f'<b>{weekday} activity:</b>'
        if query in npz.files:
            board_img = f'<img src="{name_to_file(query, weekday)}" width="200" height="150" frameborder="0">'
            K = 0.01*npz[query][day][hour]
        else:
            board_img = f'<img src="{name_to_file("""MEAN""", weekday)}" width="200" height="150" frameborder="0">'
            K = 0.01*M[day][hour]
        board_votes = f"VOTES: <b>{int(votes)}</b>"
        board = '<br>'.join([board_title, board_hours, board_votes, board_img_desc, board_img])
        
        if query in colorsep:
            fill_opacity, fill_color = 0.6, 'darkgreen'
        else:
            fill_opacity, fill_color = 0.5, 'blue'

        
        feature = {"type": "Feature",
                   "geometry": {"type": "Point", "coordinates": (lng, lat)},
                   "properties": {
                       "time": date_repr,
                       "tooltip": board,
                       "id": f"house",
                       "icon": "circle",
                       "iconstyle": {
                           "fillColor": fill_color,
                           "fillOpacity": fill_opacity,
                           "stroke": "true",
                           "radius": (votes*K+30)**(1/3),
                           "color": 'black'}}}
        features.append(feature)
    return features



In [16]:
from folium import plugins

df = pd.read_csv('bolt_restaurants_vilnius.txt')
moments = np.arange(1440, 1440*8, 60)%(1440*7)
_open, _close = readable_to_minutes(df['open']), readable_to_minutes(df['close'])
business_matrix = openness(np.transpose([_open, _close]), moments)

idx_collection = []
date_collection = []
d = {0: 'Mon', 1: 'Tue', 2: 'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
for idx, m in zip(business_matrix, moments):
    day, hour, minute = np.unravel_index(m, (7, 24, 60))
    idx_collection.append(idx)
    date_collection.append((day, hour, minute))
    
for w in range(7):
    features = []
    for i in range(w*24, (w+1)*24):
        subdf = df.loc[idx_collection[i]]
        features.extend(activity_circles(date_collection[i], subdf, colorsep=colorsep))

    m = folium.Map(location=[54.677851,25.282656], tiles='openstreetmap', zoom_start=12)
    plugins.TimestampedGeoJson(
        {"type": "FeatureCollection", "features": features},
        period="PT1H",
        add_last_point=True,
        auto_play=False,
        loop=False,
        max_speed=1,
        loop_button=True,
        date_options="HH:mm",
        time_slider_drag_update=True,
        duration="PT0H",
    ).add_to(m)

    plugins.LocateControl(auto_start=False, flyTo=True).add_to(m)
    save_route_map(m, f'bolt_animation_test_{d[w]}.html')
    IFrame(src=f'./bolt_animation_test_{d[w]}.html', width=1000, height=400)

CREATE: bolt_animation_test_Mon.html
CREATE: bolt_animation_test_Tue.html
CREATE: bolt_animation_test_Wed.html
CREATE: bolt_animation_test_Thu.html
CREATE: bolt_animation_test_Fri.html
CREATE: bolt_animation_test_Sat.html
CREATE: bolt_animation_test_Sun.html
